# Day 6 - Lab 2: Creating a Conversational Multi-Agent System (Solution)

**Objective:** Integrate the multi-agent LangGraph system from the previous lab into the FastAPI backend, creating a new `/chat` endpoint that is stateful and can handle conversational memory.

**Introduction:**
This solution notebook provides the complete code for deploying the LangGraph agent as a conversational API. It covers creating a stateless endpoint, building a Streamlit UI, and finally implementing stateful memory management.

For definitions of key terms used in this lab, please refer to the [GLOSSARY.md](../../GLOSSARY.md).

## Step 1: Setup

**Explanation:**
This setup code mocks the necessary components for the lab. We create a mock `multi_agent_app` to simulate the compiled LangGraph from Lab 6.1. This allows us to develop the API and UI without needing to run the full, complex graph. In a real application, you would import your compiled graph object here.

In [1]:
import sys
import os

# Add the project's root directory to the Python path
try:
    project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
except IndexError:
    project_root = os.path.abspath(os.path.join(os.getcwd()))

if project_root not in sys.path:
    sys.path.insert(0, project_root)

from utils import save_artifact
print("Setup complete. The following cells contain the code for your '.py' files.")

Setup complete. The following cells contain the code for your '.py' files.


## Step 2: The Challenges - Solutions

### Challenge 1 & 3: FastAPI Backend (`app/main.py`)

**Explanation:**
This code block contains the complete, final version of the backend code to be added to `app/main.py`. It includes both the stateless and stateful endpoints for comparison.

1.  **`StatefulChatRequest`**: The request model is updated to include an optional `session_id`.
2.  **Endpoint Logic**: The stateful endpoint checks if a `session_id` was provided. If not, it creates a new one, establishing a new conversation session.
3.  **`config` object**: This is the most critical part. We pass a `config` dictionary to the `.invoke()` method. LangGraph specifically looks for the `{"configurable": {"session_id": "..."}}` structure to manage memory. When it sees this, it automatically retrieves the history for that session, includes it in the context for the LLM, and saves the new turn to that same history.

In [2]:
final_backend_code = """# In app/main.py, add the following...
import uuid
from pydantic import BaseModel

# This would be your actual compiled LangGraph application
# from your_rag_system import multi_agent_app 
class MockLangGraphApp:
    def invoke(self, inputs, config):
        session_id = config.get('configurable', {}).get('session_id', 'N/A')
        print(f"Invoking agent for session {session_id} with question: {inputs.get('question')}")
        return {"answer": f"This is a stateful answer to: '{inputs.get('question')}'"}
multi_agent_app = MockLangGraphApp()

# --- Challenge 1: Stateless Endpoint ---
class ChatRequest(BaseModel):
    question: str

@app.post("/chat")
def chat_endpoint(request: ChatRequest):
    # Note the empty config dictionary, indicating a stateless call
    result = multi_agent_app.invoke({"question": request.question}, config={})
    return {"answer": result.get("answer")}

# --- Challenge 3: Stateful Endpoint ---
class StatefulChatRequest(BaseModel):
    question: str
    session_id: str | None = None

@app.post("/stateful_chat")
def stateful_chat_endpoint(request: StatefulChatRequest):
    session_id = request.session_id if request.session_id else str(uuid.uuid4())
    
    # LangGraph uses this specific config structure to manage conversational memory
    config = {"configurable": {"session_id": session_id}}
    
    inputs = {"question": request.question}
    result = multi_agent_app.invoke(inputs, config=config)
    
    return {
        "answer": result.get("answer"),
        "session_id": session_id
    }
"""
print("--- Code to add to app/main.py ---")
print(final_backend_code)

--- Code to add to app/main.py ---
# In app/main.py, add the following...
import uuid
from pydantic import BaseModel

# This would be your actual compiled LangGraph application
# from your_rag_system import multi_agent_app 
class MockLangGraphApp:
    def invoke(self, inputs, config):
        session_id = config.get('configurable', {}).get('session_id', 'N/A')
        print(f"Invoking agent for session {session_id} with question: {inputs.get('question')}")
        return {"answer": f"This is a stateful answer to: '{inputs.get('question')}'"}
multi_agent_app = MockLangGraphApp()

# --- Challenge 1: Stateless Endpoint ---
class ChatRequest(BaseModel):
    question: str

@app.post("/chat")
def chat_endpoint(request: ChatRequest):
    # Note the empty config dictionary, indicating a stateless call
    result = multi_agent_app.invoke({"question": request.question}, config={})
    return {"answer": result.get("answer")}

# --- Challenge 3: Stateful Endpoint ---
class StatefulChatRequest(Base

### Challenge 2 & 3: Streamlit UI (`chat_ui.py`)

**Explanation:**
This script creates the web UI for our chatbot. 
-   **`st.session_state`**: This is Streamlit's built-in dictionary for maintaining state across user interactions. We use it to store the chat history and the current `session_id`.
-   **Stateful Logic**: When the user sends a message, the app makes a POST request to our `/stateful_chat` endpoint. It includes the current `session_id` from `st.session_state`. When it gets a response, it updates the history and, crucially, saves the `session_id` from the response back into `st.session_state` for the next turn.

In [4]:
stateful_chat_ui_code = """import streamlit as st
import requests

st.title("STATEFUL Multi-Agent RAG Chatbot")

# Initialize session state variables
if 'history' not in st.session_state:
    st.session_state.history = []
if 'session_id' not in st.session_state:
    st.session_state.session_id = None

question = st.text_input("Ask a question about your project:", key="input_stateful")

if st.button("Send Request"):
    if question:
        # The payload now includes the session_id from our state
        payload = {"question": question, "session_id": st.session_state.session_id}
        
        try:
            response = requests.post("http://127.0.0.1:8000/stateful_chat", json=payload)
            if response.status_code == 200:
                data = response.json()
                # Store the session ID from the response for the next turn
                st.session_state.session_id = data.get('session_id') 
                answer = data.get('answer')
                st.session_state.history.append(("You", question))
                st.session_state.history.append(("Agent", answer))
            else:
                st.error(f"Failed to get response from API. Status: {response.status_code}")
        except requests.exceptions.ConnectionError as e:
            st.error(f"Could not connect to the API. Is your FastAPI server running? Error: {e}")

# Display the chat history
for author, text in st.session_state.history:
    st.write(f"**{author}:** {text}")

st.sidebar.title("Session Info")
st.sidebar.write(f"Current Session ID: {st.session_state.session_id}")
"""
save_artifact(stateful_chat_ui_code, "chat_ui.py", overwrite=True)
print("Saved 'chat_ui.py'. To run it, open your terminal and execute: streamlit run chat_ui.py")

Saved 'chat_ui.py'. To run it, open your terminal and execute: streamlit run chat_ui.py


## Lab Conclusion

Excellent! You have successfully integrated your powerful multi-agent system into a real-world application. You created a stateless API endpoint, built a UI for it, and then performed the critical upgrade to make it a stateful, conversational agent with memory. This is the complete pattern for deploying sophisticated AI assistants that can engage in natural, multi-turn dialogues with users.

> **Key Takeaway:** Stateful conversation is achieved by passing a consistent `session_id` to your agent. Frameworks like LangGraph use this ID to automatically manage and retrieve the history for each user, enabling context-aware, multi-turn conversations.